In [0]:
# Databricks notebook source
#%pip install /Volumes/mlops_dev/mtrofimo/churn_predictor/churn_predictor-0.0.2-py3-none-any.whl

# COMMAND ----------

# MAGIC %md
# MAGIC ## Feature Importance

# COMMAND ----------

import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import current_timestamp, to_utc_timestamp
import numpy as np
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import time
import os

from churn_predictor.config import ProjectConfig

spark = SparkSession.builder.getOrCreate()

# Load configuration
# Determine the environment and set the config path accordingly
if "DATABRICKS_RUNTIME_VERSION" in os.environ:
    config_path = "../project_config.yml"
else:
    config_path = os.path.abspath("project_config.yml")
config = ProjectConfig.from_yaml(config_path=config_path, env="dev")
spark = SparkSession.builder.getOrCreate()

train_set = spark.table(f"{config.catalog_name}.{config.schema_name}.train_set").toPandas()
test_set = spark.table(f"{config.catalog_name}.{config.schema_name}.test_set").toPandas()

print(train_set.columns)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Generate Synthetic Data

# COMMAND ----------
from churn_predictor.data_processor import generate_synthetic_data_with_drift

inference_data_skewed = generate_synthetic_data_with_drift(train_set, True, 200)


inference_data_skewed.display()
